In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss
warnings.filterwarnings('ignore')

# 用convert_dtypes()读取数据

In [2]:
train = pd.read_csv('./input/train.csv').convert_dtypes(convert_string=False)
test = pd.read_csv('./input/testA.csv').convert_dtypes(convert_string=False)

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [4]:
train.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,0,35000,5,19.52,917.97,E,E2,320,2 years,2,110000.0,2,2014-07-01,1,1,137,32,17.05,0,730,734,7,0,0,24178,48.9,27,0,0,Aug-2001,1,1,0,2,2,2,4,9,8,4,12,2,7,0,0,0,2
1,1,18000,5,18.49,461.90,D,D2,219843,5 years,0,46000.0,2,2012-08-01,0,0,156,18,27.83,0,700,704,13,0,0,15096,38.9,18,1,0,May-2002,1723,1,<NA>,<NA>,<NA>,<NA>,10,<NA>,<NA>,<NA>,<NA>,<NA>,13,<NA>,<NA>,<NA>,<NA>
2,2,12000,5,16.99,298.17,D,D3,31698,8 years,0,74000.0,2,2015-10-01,0,0,337,14,22.77,0,675,679,11,0,0,4606,51.8,27,0,0,May-2006,0,1,0,0,3,3,0,0,21,4,5,3,11,0,0,0,4
3,3,11000,3,7.26,340.96,A,A4,46854,10+ years,1,118000.0,1,2015-08-01,0,4,148,11,17.21,0,685,689,9,0,0,9948,52.6,28,1,0,May-1999,4,1,6,4,6,6,4,16,4,7,21,6,9,0,0,0,1
4,4,3000,3,12.99,101.07,C,C2,54,NaN,1,29000.0,2,2016-03-01,0,10,301,21,32.16,0,690,694,12,0,0,2942,32.0,27,0,0,Aug-1977,11,1,1,2,7,7,2,4,9,10,15,7,12,0,0,0,4


In [5]:
test.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14
0,800000,14000,3,10.99,458.28,B,B3,7027,10+ years,0,80000.0,0,2014-07-01,0,163,21,10.56,1,715,719,17,0,0,9846,30.7,29,0,0,Nov-1974,0,1,1,4,6,6,6,8,4,15,19,6,17,0,0,1,3
1,800001,20000,5,14.65,472.14,C,C5,60426,10+ years,0,50000.0,0,2015-07-01,2,235,8,21.40,2,670,674,5,0,0,8946,56.6,14,0,0,Jul-2001,5,1,2,1,3,3,1,1,3,3,9,3,5,0,0,2,2
2,800002,12000,3,19.99,445.91,D,D4,23547,2 years,1,60000.0,2,2016-10-01,0,526,20,33.50,0,710,714,12,0,0,970,17.6,43,1,0,Aug-2006,0,1,0,1,4,4,1,1,36,5,6,4,12,0,0,0,7
3,800003,17500,5,14.31,410.02,C,C4,636,4 years,0,37000.0,1,2014-11-01,4,248,11,13.95,0,685,689,10,1,1,10249,52.3,18,0,0,Jul-2002,4,1,0,2,2,2,4,7,2,8,14,2,10,0,0,0,3
4,800004,35000,3,17.09,1249.42,D,D1,368446,< 1 year,1,80000.0,1,2017-10-01,0,115,8,24.97,0,685,689,19,0,0,33199,35.6,22,0,0,Dec-2000,0,1,0,8,11,11,9,11,3,16,18,11,19,0,0,0,1


## 定义全局变量

# 先拼train和test 本来想封装一个函数用来拼接
因为似乎concat有个问题，会lose dtype?
复盘了一下M5 感觉M5-simple-fe，def merge-by-concat不太看得懂

### 防呆 这里+col: 'flag'，test的y('isDefault')设为-1

In [6]:
# train.shape  # (800000, 48) & len(train)=800000
# test.shape # (200000, 47)

In [7]:
TARGET = 'isDefault'
# train['flag'] = 'train'
# test['flag'] = 'test'
test[TARGET] = -1
df = pd.concat([train, test])
# df

In [8]:
# df.info()

### drop policycode, 把flag astype 'string

In [9]:
df.drop('policyCode',axis = 1, inplace=True)
df.drop('n3',axis = 1, inplace=True)

In [10]:
# df['flag'] = df['flag'].astype('string')

### issueDate

#### 这里如果string能读datetime吗? 

#### 这里有个全局变量startdate小心

In [11]:
#转化成时间格式，查了确实train的最小日期是2007-6-1，这里加了个issueDateDT,还行吧，就是日期会让数值特别大，
df['issueDate'] = pd.to_datetime(df['issueDate'],format='%Y-%m-%d')
startdate = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
# 构造时间特征
df['issueDateDT'] = df['issueDate'].apply(lambda x: x-startdate).dt.days

In [12]:
pd.set_option('display.max_rows', None)

In [13]:
# df.dtypes   # issueDate datetime64[ns]  '<M8[ns]'

### employmentLength的 years删掉，10+ <1转

#### 这里如果str是不是只能用正则

In [14]:
def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])

df['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
df['employmentLength'].replace('< 1 year', '0 years', inplace=True)
df['employmentLength'] = df['employmentLength'].apply(employmentLength_to_int)

In [15]:
df['employmentLength'].value_counts(dropna=False).sort_index()

0.0      80226
1.0      65671
2.0      90565
3.0      80163
4.0      59818
5.0      62645
6.0      46582
7.0      44230
8.0      45168
9.0      37866
10.0    328525
NaN      58541
Name: employmentLength, dtype: int64

#### 这里float，转成Int8

In [16]:
df['employmentLength'] = df['employmentLength'].astype('Int8')

### earliesCreditLine 转datetime 转year

#### 这个现在正常了，就是值有点多，不知道要怎么利用抽取信息做特征

之前用excel打开，会变成乱码

In [17]:
df['earliesCreditLine'].sample(10)

473024    Jun-2005
218824    May-1998
616199    Jun-1990
415854    Sep-2003
579362    Oct-1999
163286    May-2003
17286     Dec-1989
568809    Mar-2008
567169    Dec-1998
254051    Jan-2002
Name: earliesCreditLine, dtype: object

In [18]:
# dw 这里是提取年信息，我想利用原始信息，顺便还可以describe一下,暂时不用他的
df['earliesCreditLine'] = pd.to_datetime(df['earliesCreditLine'],format='%b-%Y')

#### +特征

In [19]:
df['creditY'] = ((df['issueDate'] - df['earliesCreditLine']).dt.days)//365

两列的时间差

In [20]:
pd.set_option('display.max_columns', None)
df.describe(include="all")

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,n0,n1,n2,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issueDateDT,creditY
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000,1000000,999999.000000,941459.000000,1000000.000000,1.000000e+06,1000000.000000,1000000,1000000.000000,1000000.000000,999999.000000,1000000.000000,999700.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,999479.000000,1.000000e+06,999342.000000,1000000.000000,1000000.000000,1000000.000000,1000000,999999.000000,949619.000000,949619.000000,949619.000000,958367.000000,949619.000000,949619.000000,949619.000000,949618.000000,949619.000000,958367.000000,912673.000000,949619.000000,949619.000000,949619.000000,1000000.000000,1000000.000000
unique,NaN,NaN,NaN,NaN,NaN,7,35,NaN,NaN,NaN,NaN,NaN,139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,NaN,NaN,B,C1,NaN,NaN,NaN,NaN,NaN,2016-03-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2001-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN,292055,63620,NaN,NaN,NaN,NaN,NaN,36542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-06-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1944-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015-10-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,499999.500000,14420.845925,3.482534,13.239673,438.105739,NaN,NaN,72091.431605,5.965504,0.614190,7.619750e+04,1.009832,NaN,-0.040390,1.745668,258.480425,16.394772,18.285902,0.317770,696.196545,700.196687,11.594484,0.215038,0.134389,1.623413e+04,51.807612,24.983451,0.416892,0.019307,NaN,1758.926990,0.511119,3.643485,5.643547,4.735022,8.105145,8.566264,8.281331,14.616502,5.593135,11.640496,0.000818,0.003431,0.089161,2.178948,2927.600845,15.804700
std,288675.278932,8720.358533,0.855704,4.765909,261.617903,NaN,NaN,106647.143239,3.690622,0.675692,7.077646e+04,0.782519,NaN,0.598313,2.367460,199.980363,11.041613,11.219721,0.879524,31.863308,31.863973,5.471342,0.606479,0.378112,2.245257e+04,24.524087,11.988140,0.493045,0.137602,NaN,7949.848213,1.329637,2.249020,3.303965,2.951852,4.800121,7.381178,4.559533,8.121567,3.217142,5.480215,0.030164,0.062495,0.508290,1.843899,591.314176,7.508422
min,0.000000,500.000000,3.000000,5.310000,14.010000,NaN,NaN,0.000000,0.000000,0.000000,0.000000e+00,0.000000,NaN,-1.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,625.000000,629.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
25%,249999.750000,8000.000000,3.000000,9.750000,248.480000,NaN,NaN,427.000000,2.000000,0.000000,4.567103e+04,0.000000,NaN,0.000000,0.000000,103.000000,8.000000,11.800000,0.000000,670.000000,674.000000,8.000000,0.000000,0.000000,5.943000e+03,33.400000,16.000000,0.000000,0.000000,NaN,0.000000,0.000000,2.000000,3.000000,3.000000,5.000000,4.000000,5.000000,9.000000,3.000000,8.000000,0.000000,0.000000,0.000000,1.000000,2587.000000,11.000000


In [21]:
# df.info()

#### 压缩一下内存

In [22]:
import psutil

## Simple "Memory profilers" to see memory usage
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2)

def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [23]:
# 这里改了一些 Int
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['Int16', 'Int32', 'Int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'Int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype('Int8')
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype('Int16')
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype('Int32')
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype('Int64')
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [24]:
print("{:>20}: {:>8}".format('Original df',sizeof_fmt(train.memory_usage(index=True).sum())))

         Original df: 315.1MiB


In [25]:
df = reduce_mem_usage(df)

Mem. usage decreased to 142.10 Mb (63.9% reduction)


In [26]:
# 看下数据类型变没变
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000000 entries, 0 to 199999
Data columns (total 47 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   id                  1000000 non-null  Int32         
 1   loanAmnt            1000000 non-null  Int32         
 2   term                1000000 non-null  Int8          
 3   interestRate        1000000 non-null  float16       
 4   installment         1000000 non-null  float16       
 5   grade               1000000 non-null  object        
 6   subGrade            1000000 non-null  object        
 7   employmentTitle     999999 non-null   Int32         
 8   employmentLength    941459 non-null   Int8          
 9   homeOwnership       1000000 non-null  Int8          
 10  annualIncome        1000000 non-null  float32       
 11  verificationStatus  1000000 non-null  Int8          
 12  issueDate           1000000 non-null  datetime64[ns]
 13  isDefault    

In [27]:
df.head()

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,n0,n1,n2,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issueDateDT,creditY
0,0,35000,5,19.515625,918.0000,E,E2,320,2,2,110000.0,2,2014-07-01,1,1,137,32,17.046875,0,730,734,7,0,0,24178,48.90625,27,0,0,2001-08-01,1,0,2,2,4,9,8,4,12,2,7,0,0,0,2,2588.0,12.0
1,1,18000,5,18.484375,462.0000,D,D2,219843,5,0,46000.0,2,2012-08-01,0,0,156,18,27.828125,0,700,704,13,0,0,15096,38.90625,18,1,0,2002-05-01,1723,<NA>,<NA>,<NA>,10,<NA>,<NA>,<NA>,<NA>,<NA>,13,<NA>,<NA>,<NA>,<NA>,1888.0,10.0
2,2,12000,5,16.984375,298.2500,D,D3,31698,8,0,74000.0,2,2015-10-01,0,0,337,14,22.765625,0,675,679,11,0,0,4606,51.81250,27,0,0,2006-05-01,0,0,0,3,0,0,21,4,5,3,11,0,0,0,4,3044.0,9.0
3,3,11000,3,7.261719,341.0000,A,A4,46854,10,1,118000.0,1,2015-08-01,0,4,148,11,17.203125,0,685,689,9,0,0,9948,52.59375,28,1,0,1999-05-01,4,6,4,6,4,16,4,7,21,6,9,0,0,0,1,2984.0,16.0
4,4,3000,3,12.992188,101.0625,C,C2,54,<NA>,1,29000.0,2,2016-03-01,0,10,301,21,32.156250,0,690,694,12,0,0,2942,32.00000,27,0,0,1977-08-01,11,1,2,7,2,4,9,10,15,7,12,0,0,0,4,3196.0,38.0


### ?? categorical:  grade

### 这里有序不知道要不要加进去

不 encode lgb读不了 这里还没跑

## be careful if using NN👇

In [49]:
for i in ['grade', 'subGrade','term', 'purpose', 'regionCode', 'postCode', 'title', 'employmentTitle']:
    df[i] = df[i].astype('category')

#### 保存

In [29]:
# df.to_csv('cf-df.csv')

In [30]:
# import pickle
# df.to_pickle('cf-df.pkl')

In [31]:
# df = pd.read_pickle('cf-df.pkl')

In [50]:
train = df[:800000]
test = df[800000:]

In [33]:
# # 如果前面用了flag
# train=train.drop('flag',axis=1,inplace=True)
# test=test.drop('flag',axis=1,inplace=True)

In [34]:
# train.head()

In [35]:
# train.info()

In [51]:
test.drop('isDefault',axis=1,inplace=True)

In [37]:
# test.info()

In [38]:
# train.to_pickle('cf-train.pkl')

In [39]:
# test.to_pickle('cf-testa.pkl')

In [40]:
# train.to_csv('cf-train.csv')

In [41]:
# test.to_csv('cf-testa.csv')

### ?? 异常值

## EDA

In [52]:
# import pandas_profiling
# pfr = pandas_profiling.ProfileReport(train)
# pfr.to_file("./traineg.html")

In [43]:
# pfr = pandas_profiling.ProfileReport(test)
# pfr.to_file("./testeg.html")

In [53]:
import pandas as pd
import numpy as np
import warnings
import os
import seaborn as sns
import matplotlib.pyplot as plt
"""
sns 相关设置
@return:
"""
# 声明使用 Seaborn 样式
sns.set()
# 有五种seaborn的绘图风格，它们分别是：darkgrid, whitegrid, dark, white, ticks。默认的主题是darkgrid。
sns.set_style("whitegrid")
# 有四个预置的环境，按大小从小到大排列分别为：paper, notebook, talk, poster。其中，notebook是默认的。
sns.set_context('talk')
# 中文字体设置-黑体
plt.rcParams['font.sans-serif'] = ['SimHei']
# 解决保存图像是负号'-'显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False
# 解决Seaborn中文显示问题并调整字体大小
sns.set(font='SimHei')

In [61]:
# train = train.fillna(-1)
# test = test.fillna(-1)

In [55]:
cat_fea = ['grade', 'subGrade', 'employmentTitle', 'verificationStatus', \
          'purpose', 'postCode', 'regionCode','applicationType','initialListStatus', 'title', 'policyCode']

In [56]:
cols = ['id', 'loanAmnt', 'term', 'interestRate', 'installment', 'grade',
       'subGrade', 'employmentTitle', 'employmentLength', 'homeOwnership',
       'annualIncome', 'verificationStatus', 'issueDate', 'isDefault',
       'purpose', 'postCode', 'regionCode', 'dti', 'detlinquency_2years',
       'ficoRangeLow', 'ficoRangeHigh', 'openAcc', 'pubRec',
       'pubRecBankruptcies', 'revolBal', 'revolUtil', 'totalAcc',
       'initialListStatus', 'applicationType', 'earliesCreditLine', 'title',
       'policyCode', 'n0', 'n1', 'n2', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9',
       'n10', 'n11', 'n12', 'n13', 'n14']

In [57]:
num_fea = [a for a in cols if a not in cat_fea]
num_fea

['id',
 'loanAmnt',
 'term',
 'interestRate',
 'installment',
 'employmentLength',
 'homeOwnership',
 'annualIncome',
 'issueDate',
 'isDefault',
 'dti',
 'detlinquency_2years',
 'ficoRangeLow',
 'ficoRangeHigh',
 'openAcc',
 'pubRec',
 'pubRecBankruptcies',
 'revolBal',
 'revolUtil',
 'totalAcc',
 'earliesCreditLine',
 'n0',
 'n1',
 'n2',
 'n4',
 'n5',
 'n6',
 'n7',
 'n8',
 'n9',
 'n10',
 'n11',
 'n12',
 'n13',
 'n14']

In [58]:
len(num_fea)

35

In [47]:
# ## 定义了一个统计函数，方便后续信息统计
# def Sta_inf(data):
#     print('_min',np.min(data))
#     print('_max:',np.max(data))
#     print('_mean',np.mean(data))
#     print('_ptp',np.ptp(data))
#     print('_std',np.std(data))
#     print('_var',np.var(data))